<a href="https://colab.research.google.com/github/Aakanksha2518/Terrorist_Detector/blob/main/Image_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving ImageDataset.zip to ImageDataset (1).zip


In [3]:
import zipfile
import os

#extract the zip file
with zipfile.ZipFile("ImageDataset.zip", "r") as zip_ref:
  zip_ref.extractall("ImageDataset")

In [4]:
#check if extracted correctly
os.listdir("ImageDataset")

['ImageDataset', '__MACOSX']

Installing Libraries & Set Up

In [5]:
import os
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

Preparing Datasets and Transforms

In [6]:
IMAGE_SIZE = 224
BATCH_SIZE = 16

import shutil

# Remove the __MACOSX folder if it exists
macosx_path = "ImageDataset/__MACOSX"
if os.path.exists(macosx_path):
    shutil.rmtree(macosx_path)
    print("Removed __MACOSX folder")

#data transformations for pretrained ResNet
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


#load dataset
dataset = datasets.ImageFolder("ImageDataset/ImageDataset", transform = transform)
class_names = dataset.classes
print("Clasees: ", class_names)

Removed __MACOSX folder
Clasees:  ['Soldier', 'Terrorist']


Splitting the Dataset

In [7]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, test_size])

#data loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)


Loading pretrained ResNet18 & modifying it

In [8]:
#Loading pretrained ResNet18
model = models.resnet18(pretrained=True)

#freeze earlier layers
for param in model.parameters():
  param.require_grad = False

#Replace final layer for 2-class output
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 195MB/s]


Defining loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.001)

Training the Model

In [10]:
Epochs = 5

for epoch in range(Epochs):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")





/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1, Loss: 0.2556
Epoch 2, Loss: 0.1600
Epoch 3, Loss: 0.1298
Epoch 4, Loss: 0.1279
Epoch 5, Loss: 0.1135


Evaluate on test set

In [11]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    # Add the number of samples in the current batch to the total count
    total += labels.size(0)
    # Accumulate the number of correctly predicted samples
    correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 96.17%
